In [1]:
import pandas as pd
import numpy as np
import heapq
import pdb

In [2]:
!ls

ce_reg_pred_0.csv	    iterate.py	     __pycache__
chi_squared.py		    Makefile	     README.md
Datafiles		    myenv	     requirements.txt
datafiles.tar.gz	    node.py	     test.bat
docs			    other_data	     tests
function_names_from_binary  parse.py
guide			    prototype.ipynb


In [3]:
df = pd.read_csv("tests/data-tiniest2/data.txt",delim_whitespace=True)

In [4]:
df.columns

Index(['target', 'num1', 'num2', 'num3'], dtype='object')

In [5]:
df['num1']

0     1.0
1     1.1
2     1.2
3     1.3
4     1.4
5     1.5
6     1.6
7     1.7
8     1.8
9     1.9
10    2.0
11    2.1
12    2.2
13    2.3
14    2.4
15    2.5
16    2.6
17    2.7
18    2.8
19    2.9
20    3.0
21    3.1
22    3.2
23    3.3
24    3.4
25    3.5
26    3.6
27    3.7
28    3.8
29    3.9
Name: num1, dtype: float64

In [6]:
for col in df.columns:
    quartile_edges= np.percentile(df[col].values, [25,50,75,100],method='linear')

In [7]:
quartile_edges

array([0.32841209, 0.55426376, 0.66929288, 0.94757109])

In [8]:
stat_pval = {'num1': 0.3992086966088876, 'num2': 7.5506181559256826e-06, 'num3' : 0.5394739470309722}
 

In [9]:
stat_pval

{'num1': 0.3992086966088876,
 'num2': 7.5506181559256826e-06,
 'num3': 0.5394739470309722}

In [10]:
top_3_keys = {key: value for key, value in stat_pval.items(
        ) if value in heapq.nsmallest(3, stat_pval.values())}

In [11]:
top_3_keys

{'num1': 0.3992086966088876,
 'num2': 7.5506181559256826e-06,
 'num3': 0.5394739470309722}

In [12]:
top_3_keys = sorted(top_3_keys.items(),key=lambda x:x[1])

In [13]:
split_var = top_3_keys[0][0]
col = split_var


In [14]:
x_uniq = df[col].drop_duplicates().sort_values()
cutpoints = x_uniq[:-1] + np.diff(x_uniq)/2


In [15]:
cutpoints

23    0.5
6     1.5
2     2.5
1     3.5
7     4.5
0     5.5
18    6.5
3     7.5
13    8.5
Name: num2, dtype: float64

In [20]:
#for cut in cutpoints:
#    right_idx = df[df[col] < cut].index.values
#    left_idx = df[df[col] >= cut].index.values
#    tot_items = len(right_idx) + len(left_idx)
#    weights = tot_items / tot_items, len(right_idx)/tot_items, len(left_idx)/tot_items
#    # Is this correct calculation of sse using the child node means to get the sse on the left and right?
#    left_resid = df.loc[left_idx, self.tgt] - df.loc[left_idx, self.tgt].mean()
#    right_resid = df.loc[right_idx, self.tgt] - df.loc[right_idx, self.tgt].mean()
#    node_resid = df[self.tgt].values - node.y_mean
#    tot_sse = weights[0]*_sse(node_resid) - weights[1] * \
#        _sse(right_resid) - weights[2]*_sse(left_resid)
#    if smallest_tot_sse == None or smallest_tot_sse > tot_sse:
#        smallest_tot_sse = tot_sse
#        cut_with_smallest_sse = cut
old_df = df
df = df.loc[df['num2'] <= 4.5]
greatest_tot_sse = None
node_sse = ((df.target - df.target.mean())**2).sum()
best_cut = None
for cut in cutpoints:
    right_idx = df[df[col] > cut].index
    left_idx = df.drop(right_idx, axis=0).index
    left_mean = df.loc[left_idx].target.mean()
    right_mean = df.loc[right_idx].target.mean()
    tot_items = len(left_idx) + len(right_idx)
    #left_sse = ((df.loc[left_idx].target - left_mean)**2).sum()
    #right_sse = ((df.loc[right_idx].target - right_mean)**2).sum()
    nAL = len(left_idx)
    nAR = len(right_idx)
    cut_sse = (nAL * nAR / tot_items) * (left_mean - right_mean)**2
    #weights = 1, len(left_idx) / tot_items, len(right_idx) / tot_items
    #cut_sse = weights[0]*node_sse - weights[1]*left_sse - weights[2]*right_sse
    print(cut, cut_sse)
    if greatest_tot_sse == None or cut_sse > greatest_tot_sse:
        greatest_tot_sse = cut_sse
        best_cut = cut
        
        

0.5 19.766722689075632
1.5 53.37529411764708
2.5 54.372794117647054
3.5 41.339294117647036
4.5 nan
5.5 nan
6.5 nan
7.5 nan
8.5 nan


In [21]:
best_cut

2.5

In [18]:
len(df)

30

In [19]:
df

,target,num1,num2,num3
0,11.0,1.0,5,0.089177
1,7.1,1.1,3,0.665040
2,5.2,1.2,2,0.935025
3,15.3,1.3,7,0.670711
4,11.4,1.4,5,0.653010
5,11.5,1.5,5,0.478610
6,3.6,1.6,1,0.623726
7,9.7,1.7,4,0.111696
8,7.8,1.8,3,0.887405
9,3.9,1.9,1,0.312382
